In [234]:
import pip
pip.main(['install', 'catboost'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [294]:
import json
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

In [295]:
!pip freeze > requirements.txt

In [296]:
RANDOM_SEED = 42

In [297]:
VAL_SIZE   = 0.20   # 20%

In [323]:
df_train = pd.read_csv('./input/dataset_with_electro.csv', usecols = lambda x: x not in ('photos_links', 'descr'))
df_test = pd.read_csv('./input/test.csv')

C:\Users\allkotraz\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5,6,7,8,12,13,18,25,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [324]:
pd.set_option('display.max_columns', None)

In [325]:
df_train.sample(5)

,brand,model,generation,date_posted,views_total,views_today,offer_id,year,mileage,body_type,color,fuel_type,engine_volume,engine_power,transmission,drive,wheel,state,owners_count,pts,customs,exchange,vin,license_plate,photos_count,complectation,price,url,catalog_link,orig_url
157433,Продажа автомобилей,Chevrolet,I,12 июня 2019,759,2,1088857892,2008,180000,седан,серый,Бензин,1.5,86,механическая,передний,Левый,Не требует ремонта,2,Оригинал,Растаможен,NaN,Y6D**************,******|199,1,NaN,120000,https://auto.ru/cars/used/sale/chevrolet/lanos...,https://auto.ru/catalog/cars/chevrolet/lanos/2...,https://auto.ru/cars/used/sale/chevrolet/lanos...
36537,Продажа автомобилей,LADA (ВАЗ),I Рестайлинг,12 мая,3256,2,1.10351e+09,2003,91000,внедорожник 3 дв.,пурпурный,Бензин,1.7,79,механическая,полный,Левый,Не требует ремонта,1,Оригинал,Растаможен,NaN,XTA**************,******|190,6,NaN,140000,https://auto.ru/cars/used/sale/vaz/2121/110351...,https://auto.ru/catalog/cars/vaz/2121/2307327/...,https://auto.ru/cars/used/sale/vaz/2121/110351...
526989,Продажа автомобилей,Hyundai,I,22 апреля,930,10,1.10329e+09,2013,115000,седан,белый,Бензин,1.4,107,механическая,передний,Левый,Не требует ремонта,2,Оригинал,Растаможен,NaN,Z94**************,******|161,27,NaN,490000,https://auto.ru/cars/used/sale/hyundai/solaris...,https://auto.ru/catalog/cars/hyundai/solaris/6...,https://auto.ru/cars/used/sale/hyundai/solaris...
143188,Продажа автомобилей,Hyundai,I Рестайлинг,9 августа,70,1,1104862053,2016,72000,седан,серый,Бензин,1.6,123,автоматическая,передний,Левый,Не требует ремонта,2,Оригинал,Растаможен,NaN,Z94**************,******|123,1,"Мультифункциональное рулевое колесо,Система до...",795000,https://auto.ru/cars/used/sale/hyundai/solaris...,https://auto.ru/catalog/cars/hyundai/solaris/2...,https://auto.ru/cars/used/sale/hyundai/solaris...
491484,Продажа автомобилей,Chevrolet,I,15 мая,222,2,1.10355e+09,2008,160000,седан,серебристый,Бензин,1.5,86,механическая,передний,Левый,Не требует ремонта,1,Оригинал,Растаможен,NaN,Y6D**************,******|102,1,NaN,126599,https://auto.ru/cars/used/sale/chevrolet/lanos...,https://auto.ru/catalog/cars/chevrolet/lanos/2...,https://auto.ru/cars/used/sale/chevrolet/lanos...


In [326]:
df_test.sample(5)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
25387,седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/jett...,золотистый,NaN,"Автомобиль в идеальном состоянии, не требует а...",1.4 LTR,122 N12,NaN,бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,188614,2010,"{""code"":""JETTA"",""name"":""Jetta"",""ru_name"":""Джет...",JETTA,1.4 AMT (122 л.с.),4,1603221223,RUB,2012,1101213745,"{""id"":""7720682"",""displacement"":1390,""engine_ty...",SEDAN ROBOT 1.4,роботизированная,EUROPEAN,2 владельца,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
18166,внедорожник 5 дв.,MERCEDES,https://auto.ru/cars/used/sale/mercedes/g_klas...,чёрный,NaN,Комфорт: 5. Салон: 3. Безопасность: 4. Обзор: ...,3.0 LTR,245 N12,"{""cruise-control"":true,""tinted-glass"":true,""ai...",дизель,https://autoru.naydex.net/mcQh17801/f1b5751dCi...,94574,2015,"{""code"":""G_KLASSE"",""name"":""G-Класс"",""ru_name"":...",G_KLASSE,350 d 3.0d AT (245 л.с.) 4WD,5,1603250649,RUB,2016,1101035002,"{""id"":""20632445"",""name"":""350"",""nameplate"":""350...",ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
8609,внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x5_m/110089...,чёрный,NaN,"Всех предыдущих владельцев знаю , по факту я 4...",4.4 LTR,555 N12,NaN,бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,99000,2009,"{""code"":""X5_M"",""name"":""X5 M"",""ru_name"":""Х5 М"",...",X5_M,4.4 AT (555 л.с.) 4WD,5,1603111829,RUB,2009,1100894554,"{""id"":""4731941"",""displacement"":4395,""engine_ty...",ALLROAD_5_DOORS AUTOMATIC 4.4,автоматическая,EUROPEAN,3 или более,NaN,Дубликат,полный,Левый,Не требует ремонта,Растаможен
34544,внедорожник 5 дв.,TOYOTA,https://auto.ru/cars/used/sale/toyota/land_cru...,серый,NaN,В связи с покупкой недвижимости продаю автомо...,2.8 LTR,177 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",дизель,https://autoru.naydex.net/16IvE9y79/a7f8demo/N...,11882,2017,"{""code"":""LAND_CRUISER_PRADO"",""name"":""Land Crui...",LAND_CRUISER_PRADO,Arctic Trucks AT38 2.8d AT (177 л.с.) 4WD,5,1603690667,RUB,2019,1101367483,"{""id"":""21500259"",""name"":""Arctic Trucks"",""namep...",ALLROAD_5_DOORS AUTOMATIC 2.8,автоматическая,JAPANESE,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
6454,седан,VOLVO,https://auto.ru/cars/used/sale/volvo/940/11012...,синий,NaN,"На ходу, дефекты по кузову",2.3 LTR,131 N12,NaN,бензин,https://autoru.naydex.net/ESooa9149/468b9eoMem...,347000,1988,"{""code"":""940"",""name"":""940"",""ru_name"":""940"",""mo...",940,2.3 MT (131 л.с.),4,1603236110,RUB,1991,1101298685,"{""id"":""8226671"",""displacement"":2316,""engine_ty...",SEDAN MECHANICAL 2.3,механическая,EUROPEAN,3 или более,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен


In [327]:
df_train.brand.unique()

array(['Продажа автомобилей', nan, 'brand'], dtype=object)

In [328]:
#df_train = df_train.dropna(subset=['price'])
df_train = df_train.drop(columns = ['brand'])

In [329]:
df_train.isna().sum()

model                66
generation          100
date_posted          66
views_total       22792
views_today       22792
offer_id             66
year                 66
mileage              66
body_type           100
color                66
fuel_type           102
engine_volume       102
engine_power        102
transmission        102
drive               102
wheel                66
state                66
owners_count        101
pts                  86
customs              66
exchange         413147
vin               15956
license_plate    120300
photos_count          0
complectation    218559
price              7438
url                   0
catalog_link        102
orig_url              0
dtype: int64

In [330]:
df_train.dropna(subset=['body_type','engine_volume','engine_power','fuel_type','transmission','owners_count', 'pts', 'drive'], inplace=True)

In [331]:
df_test['engineDisplacement'] = df_test['engineDisplacement'].apply(lambda x: float(x.split()[0]) if x!=' LTR' else 0)
df_test['enginePower'] = df_test['enginePower'].apply(lambda x: float(x.split()[0]))
df_test['productionDate'] = df_test['productionDate'].astype(int)
df_test = df_test.rename(columns={'ПТС': 'pts',
                                  'Таможня': 'customs',
                                  'Состояние': 'state',
                                  'Руль': 'wheel',
                                  'Привод': 'drive',
                                  'Владельцы': 'owners_count',
                                  'engineDisplacement': 'engine_volume',
                                  'fuelType': 'fuel_type',
                                  'enginePower': 'engine_power',
                                  'productionDate': 'year',
                                  'vehicleTransmission': 'transmission',
                                  'brand': 'model',
                                  'bodyType': 'body_type'
                                 
                                 })


In [335]:
df_train['price'] = pd.to_numeric(df_train['price'], errors='coerce')
y = df_train['price']

In [336]:
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

columns = ['body_type',
           'model',
           'color',
           'engine_volume',
           'engine_power',
           'fuel_type',
           'mileage',
           'year',
           'transmission',
           'owners_count',
           'pts',
           'drive',
           'wheel',
           'state',
           'customs',
           'sample'
          ]
df_train = df_train[columns]
df_test = df_test[columns]

In [337]:
df_test.sample(5)

,body_type,model,color,engine_volume,engine_power,fuel_type,mileage,year,transmission,owners_count,pts,drive,wheel,state,customs,sample
29087,компактвэн,MITSUBISHI,серый,1.2,91.0,бензин,123000,2014,вариатор,3 или более,Оригинал,передний,Правый,Не требует ремонта,Растаможен,0
24719,фургон,VOLKSWAGEN,синий,2.0,140.0,дизель,129000,2011,механическая,1 владелец,Оригинал,передний,Левый,Не требует ремонта,Растаможен,0
21402,внедорожник 5 дв.,TOYOTA,чёрный,4.7,235.0,бензин,386000,2003,автоматическая,2 владельца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,0
6946,внедорожник 5 дв.,VOLVO,бежевый,2.5,210.0,бензин,361501,2008,автоматическая,2 владельца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,0
20961,хэтчбек 5 дв.,TOYOTA,красный,1.3,86.0,бензин,320000,1999,механическая,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,0


In [338]:
df_test.owners_count.unique()

array(['3 или более', '1\xa0владелец', '2\xa0владельца'], dtype=object)

In [339]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [340]:
df_train.model.unique()

array(['Jeep', 'Nissan', 'LADA (ВАЗ)', 'Hyundai', 'Renault', 'Toyota',
       'Kia', 'Suzuki', 'Chevrolet', 'Volvo', 'Peugeot', 'Ford', 'ГАЗ',
       'Mercedes-Benz', 'Audi', 'Mitsubishi', 'BMW', 'Volkswagen', 'Opel',
       'Mazda', 'ЗАЗ', 'Dodge', 'Porsche', 'УАЗ', 'Great Wall', 'Skoda',
       'Honda', 'Cadillac', 'Subaru', 'Infiniti', 'Citroen', 'Москвич',
       'Ferrari', 'Chery', 'Lifan', 'Datsun', 'Daewoo', 'MINI', 'Lexus',
       'Geely', 'Land Rover', 'Rover', 'Plymouth', 'SsangYong', 'TATA',
       'ИЖ', 'Ravon', 'Isuzu', 'Daihatsu', 'GMC', 'Fiat', 'JAC',
       'Lincoln', 'Saab', 'Bentley', 'Tesla', 'Genesis', 'HuangHai',
       'Chrysler', 'Jaguar', 'Changan', 'Tianye', 'Brilliance', 'Haval',
       'SEAT', 'Pontiac', 'ЛуАЗ', 'Hafei', 'Smart', 'BYD', 'Acura',
       'Zotye', 'Maybach', 'Alfa Romeo', 'Saturn', 'ТагАЗ', 'Trabant',
       'RAM', 'Haima', 'Vortex', 'Mercury', 'Xpeng', 'Hummer',
       'Lamborghini', 'Derways', 'Rolls-Royce', 'BAIC', 'Lancia', 'Dacia',
       '

In [341]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570008 entries, 0 to 570007
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   body_type      570008 non-null  object
 1   model          570008 non-null  object
 2   color          570008 non-null  object
 3   engine_volume  570008 non-null  object
 4   engine_power   570008 non-null  object
 5   fuel_type      570008 non-null  object
 6   mileage        570008 non-null  object
 7   year           570008 non-null  object
 8   transmission   570008 non-null  object
 9   owners_count   570008 non-null  object
 10  pts            570007 non-null  object
 11  drive          570008 non-null  object
 12  wheel          570008 non-null  object
 13  state          570008 non-null  object
 14  customs        570008 non-null  object
 15  sample         570008 non-null  int64 
dtypes: int64(1), object(15)
memory usage: 69.6+ MB


In [342]:
data.isna().sum()

body_type        0
model            0
color            0
engine_volume    0
engine_power     0
fuel_type        0
mileage          0
year             0
transmission     0
owners_count     0
pts              1
drive            0
wheel            0
state            0
customs          0
sample           0
dtype: int64

In [343]:
data.dropna(subset=['pts'], inplace=True)

In [344]:
for colum in ['body_type', 'model', 'color', 'engine_volume', 'fuel_type', 'transmission', 'pts', 'drive', 'wheel', 'state', 'customs']:
    data[colum] = data[colum].astype('category').cat.codes

In [345]:
data['engine_power'] = pd.to_numeric(data['engine_power'], errors='coerce')
data['mileage'] = pd.to_numeric(data['mileage'], errors='coerce')
data['year'] = pd.to_numeric(data['year'], errors='coerce')
data['owners_count'] = pd.to_numeric(data['owners_count'], errors='coerce')



In [346]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

In [347]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [348]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [355]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [356]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engine_volume'].map(tmp_train.groupby('engine_volume')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

Точность наивной модели по метрике MAPE: 133.69%


In [357]:
predict.values

array([520000., 510000., 520000., ..., 520000., 515000., 510000.])

In [349]:
data

,body_type,model,color,engine_volume,engine_power,fuel_type,mileage,year,transmission,owners_count,pts,drive,wheel,state,customs,sample
0,9,122,14,9,105.0,10,74000.0,2014.0,4,NaN,2,2,1,2,2,0
1,9,122,16,13,110.0,10,60563.0,2017.0,3,NaN,2,2,1,2,2,0
2,9,122,13,15,152.0,10,88000.0,2014.0,4,NaN,2,2,1,2,2,0
3,9,122,7,13,110.0,10,95000.0,2014.0,1,NaN,2,2,1,2,2,0
4,9,122,2,15,152.0,10,58536.0,2012.0,1,NaN,2,2,1,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570003,5,29,5,13,120.0,1,280343.0,2011.0,3,3.0,2,2,1,2,2,1
570004,1,77,5,14,79.0,1,184000.0,2008.0,3,2.0,2,3,1,2,2,1
570005,2,147,2,17,170.0,1,193000.0,2013.0,1,2.0,2,3,1,2,2,1
570006,16,54,2,30,370.0,1,149696.0,2017.0,1,3.0,2,3,1,2,2,1


In [358]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/libs/metrics/metric.cpp:6477: metric/loss-function RMSE do not allows nan value on target

In [359]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.

In [7]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535479 entries, 0 to 535478
Data columns (total 30 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   brand          535413 non-null  object
 1   model          535413 non-null  object
 2   generation     535379 non-null  object
 3   date_posted    535413 non-null  object
 4   views_total    512687 non-null  object
 5   views_today    512687 non-null  object
 6   offer_id       535413 non-null  object
 7   year           535413 non-null  object
 8   mileage        535413 non-null  object
 9   body_type      535379 non-null  object
 10  color          535413 non-null  object
 11  fuel_type      535377 non-null  object
 12  engine_volume  535377 non-null  object
 13  engine_power   535377 non-null  object
 14  transmission   535377 non-null  object
 15  drive          535377 non-null  object
 16  wheel          535413 non-null  object
 17  state          535413 non-null  object
 18  owne

In [8]:
df.isna().sum()

brand                66
model                66
generation          100
date_posted          66
views_total       22792
views_today       22792
offer_id             66
year                 66
mileage              66
body_type           100
color                66
fuel_type           102
engine_volume       102
engine_power        102
transmission        102
drive               102
wheel                66
state                66
owners_count        101
pts                  86
customs              66
exchange         413147
vin               15956
license_plate    120300
photos_count          0
complectation    218559
price              7438
url                   0
catalog_link        102
orig_url              0
dtype: int64

In [9]:
df = df.dropna(subset=['price'])

In [10]:
df = df.dropna(subset = ['body_type'])

In [11]:
df.dropna(subset=['views_total', 'views_today'], inplace = True)
df.drop(columns = ['views_today'], inplace = True)

In [12]:
df.isna().sum()

brand                 0
model                 0
generation            0
date_posted           0
views_total           0
offer_id              0
year                  0
mileage               0
body_type             0
color                 0
fuel_type             2
engine_volume         2
engine_power          2
transmission          2
drive                 2
wheel                 0
state                 0
owners_count         32
pts                  19
customs               0
exchange         389817
vin               15044
license_plate    113154
photos_count          0
complectation    206216
price                 0
url                   0
catalog_link          2
orig_url              0
dtype: int64

In [30]:
df = df.dropna(subset=['owners_count'])

In [31]:
df.isna().sum()

brand                 0
model                 0
generation            0
date_posted           0
views_total           0
offer_id              0
year                  0
mileage               0
body_type             0
color                 0
fuel_type             2
engine_volume         2
engine_power          2
transmission          2
drive                 2
wheel                 0
state                 0
owners_count          0
pts                  19
customs               0
exchange         389792
vin               15025
license_plate    113124
photos_count          0
complectation    206186
price                 0
url                   0
catalog_link          2
orig_url              0
dtype: int64

In [40]:
df['owners_count'] = pd.to_numeric(df['owners_count'], errors='coerce')

In [49]:
df.owners_count.unique()

array([3., 2., 1.])

In [48]:
df = df.dropna(subset=['owners_count'])

In [43]:
df.loc[(df.pts.isna()==True) & (df.owners_count<3), 'pts'] = 'Оригинал'
df.loc[(df.pts.isna()==True) & (df.owners_count>=3), 'pts'] = 'Дубликат'

In [44]:
df.exchange.unique()

array([nan, 'Рассмотрю варианты', 'exchange'], dtype=object)

In [45]:
df['exchange'] = df.exchange.apply(lambda x: 1 if x == 'Рассмотрю варианты' else 0)

In [50]:
df.isna().sum()

brand                 0
model                 0
generation            0
date_posted           0
views_total           0
offer_id              0
year                  0
mileage               0
body_type             0
color                 0
fuel_type             2
engine_volume         2
engine_power          2
transmission          2
drive                 2
wheel                 0
state                 0
owners_count          0
pts                   0
customs               0
exchange              0
vin               15025
license_plate    113124
photos_count          0
complectation    206186
price                 0
url                   0
catalog_link          2
orig_url              0
dtype: int64

In [51]:
df = df.dropna(subset = ['vin'])
df = df.drop(columns = ['vin'])

In [52]:
df.isna().sum()

brand                 0
model                 0
generation            0
date_posted           0
views_total           0
offer_id              0
year                  0
mileage               0
body_type             0
color                 0
fuel_type             0
engine_volume         0
engine_power          0
transmission          0
drive                 0
wheel                 0
state                 0
owners_count          0
pts                   0
customs               0
exchange              0
license_plate    106571
photos_count          0
complectation    195236
price                 0
url                   0
catalog_link          0
orig_url              0
dtype: int64

In [53]:
df.drop(columns = ['license_plate'], inplace = True)

In [54]:
df['complectation'] = df.complectation.fillna(value = 'undefined')